In [1]:
import pandas as pd 
import numpy as np 

from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.neural_network import MLPRegressor
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_excel("preparedData.xlsx")

In [3]:
data.head(25)

,date,new_cases,new_cases_smoothed,reproduction_rate,new_tests,positive_rate,tests_per_case,total_vaccinations,people_vaccinated,people_fully_vaccinated,...,book covid test: (Ireland)_y,covid test dublin: (Ireland),covid test centre: (Ireland),hse covid vaccine: (Ireland),hse vaccine portal: (Ireland),hse portal vaccine: (Ireland),pcr test hse: (Ireland),hse covid test: (Ireland),hse vaccine registration: (Ireland),how long will it take to vaccinate ireland: (Ireland)
0,2020-03-15,0,15.714,2.51,0,0.0000,0.0,0,0,0,...,0.0,1.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
1,2020-03-16,40,21.143,2.59,0,0.0000,0.0,0,0,0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2020-03-17,54,27.000,2.70,0,0.0000,0.0,0,0,0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,2020-03-18,69,35.571,2.78,0,0.0000,0.0,0,0,0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,2020-03-19,265,73.429,2.84,946,0.0000,0.0,0,0,0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,2020-03-20,126,84.714,2.65,1469,0.0000,0.0,0,0,0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6,2020-03-21,102,93.714,2.49,1564,0.0000,0.0,0,0,0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7,2020-03-22,121,111.000,2.39,978,0.0000,0.0,0,0,0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8,2020-03-23,219,136.571,2.28,2581,0.0000,0.0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,2020-03-24,204,158.000,2.17,1566,0.0000,0.0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [4]:
data['date'] = pd.to_datetime(data['date'])
data.sort_values(by='date', ascending=True, inplace=True)
data = data.set_index('date')

In [5]:
data.drop(columns=['tests_per_case', 'new_cases','new_tests',
            'people_vaccinated', 'people_fully_vaccinated', 'total_boosters',
       'new_vaccinations','population',
       'population_density', 'median_age', 'aged_65_older', 'aged_70_older',
       'gdp_per_capita', 'extreme_poverty', 'cardiovasc_death_rate',
       'diabetes_prevalence', 'female_smokers', 'male_smokers',
       'life_expectancy', 'human_development_index', 
       'COVID-19 testing: (Ireland)', 'COVID-19 rapid antigen test: (Ireland)',
       'Health Service Executive: (Ireland)', 'Vaccination: (Ireland)',
       'book covid test: (Ireland)_x', 'how many covid cases today: (Ireland)',
       'pcr covid test: (Ireland)', 'close contact covid: (Ireland)',
       'book a covid test: (Ireland)', 'vaccination centre: (Ireland)',
       'pharmacy near me: (Ireland)',
       'Treatment and management of COVID-19: (Ireland)',
       'Hand sanitizer: (Ireland)', 'Face mask: (Ireland)',
       'book covid test: (Ireland)_y', 'covid test dublin: (Ireland)',
       'covid test centre: (Ireland)', 'hse covid vaccine: (Ireland)',
       'hse vaccine portal: (Ireland)', 'hse portal vaccine: (Ireland)',
       'pcr test hse: (Ireland)', 'hse covid test: (Ireland)',
       'hse vaccine registration: (Ireland)',
       'how long will it take to vaccinate ireland: (Ireland)'],inplace=True)

In [6]:
data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
new_cases_smoothed,702.0,1.765405e+03,3.401510e+03,6.571,286.4645,536.21400,1.522107e+03,2.375200e+04
reproduction_rate,702.0,1.076068e+00,3.489241e-01,0.000,0.8800,1.02000,1.250000e+00,2.840000e+00
positive_rate,702.0,7.688519e-02,9.587108e-02,0.000,0.0228,0.04015,8.922500e-02,5.523000e-01
total_vaccinations,702.0,2.886069e+06,3.631357e+06,0.000,0.0000,447297.00000,6.680368e+06,1.050748e+07
stringency_index,702.0,6.209577e+01,1.978799e+01,0.000,44.4400,55.56000,8.148000e+01,9.074000e+01
covid: (Ireland),701.0,2.971897e+01,1.361625e+01,15.000,21.0000,26.00000,3.400000e+01,1.000000e+02


In [7]:
df = data.copy(deep=True)

In [8]:
df.loc[:,'ncs'] = df.loc[:,'new_cases_smoothed'].shift(5)
df.loc[:,'ncs_diff'] = df.loc[:,'ncs'].diff()
df.loc[:,'ncs2'] = df.loc[:,'ncs'].shift()
df.loc[:,'ncs_diff2'] = df.loc[:,'ncs2'].diff()
df.loc[:,'ncs3'] = df.loc[:,'ncs2'].shift()
df.loc[:,'ncs_diff3'] = df.loc[:,'ncs3'].diff()
df.loc[:,'ncs4'] = df.loc[:,'ncs3'].shift()
df.loc[:,'ncs_diff4'] = df.loc[:,'ncs4'].diff()
df.loc[:,'ncs5'] = df.loc[:,'ncs4'].shift()
df.loc[:,'ncs_diff5'] = df.loc[:,'ncs5'].diff()
df.loc[:,'ncs6'] = df.loc[:,'ncs5'].shift()
df.loc[:,'ncs_diff6'] = df.loc[:,'ncs6'].diff()
df.loc[:,'ncs7'] = df.loc[:,'ncs6'].shift()
df.loc[:,'ncs_diff7'] = df.loc[:,'ncs7'].diff()

df.loc[:,'rr'] = df.loc[:,'reproduction_rate'].shift()
df.loc[:,'rr_diff'] = df.loc[:,'rr'].diff()
df.loc[:,'rr2'] = df.loc[:,'rr'].shift()
df.loc[:,'rr_diff2'] = df.loc[:,'rr2'].diff()
df.loc[:,'rr3'] = df.loc[:,'rr2'].shift()
df.loc[:,'rr_diff3'] = df.loc[:,'rr3'].diff()
df.loc[:,'rr4'] = df.loc[:,'rr3'].shift()
df.loc[:,'rr_diff4'] = df.loc[:,'rr4'].diff()
df.loc[:,'rr5'] = df.loc[:,'rr4'].shift()
df.loc[:,'rr_diff5'] = df.loc[:,'rr5'].diff()
df.loc[:,'rr6'] = df.loc[:,'rr5'].shift()
df.loc[:,'rr_diff6'] = df.loc[:,'rr6'].diff()
df.loc[:,'rr7'] = df.loc[:,'rr6'].shift()
df.loc[:,'rr_diff7'] = df.loc[:,'rr7'].diff()

df.loc[:,'pr'] = df.loc[:,'positive_rate'].shift(7)
df.loc[:,'pr_diff'] = df.loc[:,'pr'].diff()
df.loc[:,'pr2'] = df.loc[:,'pr'].shift()
df.loc[:,'pr_diff2'] = df.loc[:,'pr2'].diff()
df.loc[:,'pr3'] = df.loc[:,'pr2'].shift()
df.loc[:,'pr_diff3'] = df.loc[:,'pr3'].diff()
df.loc[:,'pr4'] = df.loc[:,'pr3'].shift()
df.loc[:,'pr_diff4'] = df.loc[:,'pr4'].diff()
df.loc[:,'pr5'] = df.loc[:,'pr4'].shift()
df.loc[:,'pr_diff5'] = df.loc[:,'pr5'].diff()
df.loc[:,'pr6'] = df.loc[:,'pr5'].shift()
df.loc[:,'pr_diff6'] = df.loc[:,'pr6'].diff()
df.loc[:,'pr7'] = df.loc[:,'pr6'].shift()
df.loc[:,'pr_diff7'] = df.loc[:,'pr7'].diff()

df.loc[:,'tv'] = df.loc[:,'total_vaccinations'].shift()
df.loc[:,'tv_diff'] = df.loc[:,'tv'].diff()
df.loc[:,'tv2'] = df.loc[:,'tv'].shift()
df.loc[:,'tv_diff2'] = df.loc[:,'tv2'].diff()
df.loc[:,'tv3'] = df.loc[:,'tv2'].shift()
df.loc[:,'tv_diff3'] = df.loc[:,'tv3'].diff()
df.loc[:,'tv4'] = df.loc[:,'tv3'].shift()
df.loc[:,'tv_diff4'] = df.loc[:,'tv4'].diff()
df.loc[:,'tv5'] = df.loc[:,'tv4'].shift()
df.loc[:,'tv_diff5'] = df.loc[:,'tv5'].diff()
df.loc[:,'tv6'] = df.loc[:,'tv5'].shift()
df.loc[:,'tv_diff6'] = df.loc[:,'tv6'].diff()
df.loc[:,'tv7'] = df.loc[:,'tv6'].shift()
df.loc[:,'tv_diff7'] = df.loc[:,'tv7'].diff()

In [9]:
df.loc[:,'si'] = df.loc[:,'stringency_index'].shift(7)
df.loc[:,'si_diff'] = df.loc[:,'si'].diff()
df.loc[:,'si2'] = df.loc[:,'si'].shift()
df.loc[:,'si_diff2'] = df.loc[:,'si2'].diff()
df.loc[:,'si3'] = df.loc[:,'si2'].shift()
df.loc[:,'si_diff3'] = df.loc[:,'si3'].diff()
df.loc[:,'si4'] = df.loc[:,'si3'].shift()
df.loc[:,'si_diff4'] = df.loc[:,'si4'].diff()
df.loc[:,'si5'] = df.loc[:,'si4'].shift()
df.loc[:,'si_diff5'] = df.loc[:,'si5'].diff()
df.loc[:,'si6'] = df.loc[:,'si5'].shift()
df.loc[:,'si_diff6'] = df.loc[:,'si6'].diff()
df.loc[:,'si7'] = df.loc[:,'si6'].shift()
df.loc[:,'si_diff7'] = df.loc[:,'si7'].diff()

In [10]:
df.loc[:,'google'] = df.loc[:,'covid: (Ireland)'].shift()
df.loc[:,'google_diff'] = df.loc[:,'google'].diff()
df.loc[:,'google2'] = df.loc[:,'google'].shift()
df.loc[:,'google_diff2'] = df.loc[:,'google2'].diff()
df.loc[:,'google3'] = df.loc[:,'google2'].shift()
df.loc[:,'google_diff3'] = df.loc[:,'google3'].diff()
df.loc[:,'google4'] = df.loc[:,'google3'].shift()
df.loc[:,'google_diff4'] = df.loc[:,'google4'].diff()
df.loc[:,'google5'] = df.loc[:,'google4'].shift()
df.loc[:,'google_diff5'] = df.loc[:,'google5'].diff()
df.loc[:,'google6'] = df.loc[:,'google5'].shift()
df.loc[:,'google_diff6'] = df.loc[:,'google6'].diff()
df.loc[:,'google7'] = df.loc[:,'google6'].shift()
df.loc[:,'google_diff7'] = df.loc[:,'google7'].diff()

In [11]:
df = df.fillna(0)

In [12]:
df.head(5)

,new_cases_smoothed,reproduction_rate,positive_rate,total_vaccinations,stringency_index,covid: (Ireland),ncs,ncs_diff,ncs2,ncs_diff2,...,google3,google_diff3,google4,google_diff4,google5,google_diff5,google6,google_diff6,google7,google_diff7
date,,,,,,,,,,,,,,,,,,,,,
2020-03-15,15.714,2.51,0.0,0,48.15,50.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-16,21.143,2.59,0.0,0,48.15,56.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-17,27.000,2.70,0.0,0,48.15,56.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-18,35.571,2.78,0.0,0,48.15,56.0,0.0,0.0,0.0,0.0,...,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-19,73.429,2.84,0.0,0,48.15,56.0,0.0,0.0,0.0,0.0,...,56.0,6.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
df.shape

(702, 48)

In [14]:
n = len(df)

In [15]:
variables = df.columns.drop(['new_cases_smoothed','ncs'])

In [16]:
X_train = df.iloc[0:int(n*0.7)].drop(columns=['new_cases_smoothed','ncs'])
y_train = df.iloc[0:int(n*0.7)].drop(columns=variables, axis=1)

In [17]:
X_test = df.iloc[int(n*0.9):].drop(columns=['new_cases_smoothed','ncs'])
y_test = df.iloc[int(n*0.9):].drop(columns=variables, axis=1)

In [18]:
X_val = df.iloc[int(n*0.7):int(n*0.9)].drop(columns=['new_cases_smoothed','ncs'])
y_val = df.iloc[int(n*0.7):int(n*0.9)].drop(columns=variables, axis=1)

In [19]:
model = MLPRegressor()

In [20]:
param_search = {
    "hidden_layer_sizes": [(1,),(50,),(100,),(150,),(200,)],
    "activation": ["identity", "logistic", "tanh", "relu"],
    "solver": ["lbfgs", "sgd", "adam"],
    "alpha": [0.00005,0.0005, 0.005],
    "learning_rate": ['constant', 'invscaling', 'adaptive']
}

In [21]:
tsvc = TimeSeriesSplit(n_splits=5)
gsearch = GridSearchCV(estimator=model, cv=tsvc, param_grid=param_search, scoring='r2')
gsearch.fit(X_train, y_train)
best_score = gsearch.best_score_
best_model = gsearch.best_estimator_

In [22]:
print(best_model)

MLPRegressor(activation='identity', alpha=0.0005, hidden_layer_sizes=(1,),
             learning_rate='adaptive', solver='lbfgs')


In [23]:
print(best_score)

0.6324688135718454


In [24]:
from sklearn.metrics import r2_score

y_pred = best_model.predict(X_test)

print(r2_score(y_test, y_pred))

0.8151902508551438


In [25]:
y_val_pred = best_model.predict(X_val)

In [26]:
print(r2_score(y_val, y_val_pred))

0.876443302244575
